Цель: Ознакомится с моделями классификации и регрессии.

In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold

После импорта библиотек, считываем данные с файла.

In [48]:
#считывание данных с файла
df = pd.read_csv('vgsales_20(19).csv')
df

,Rank,Name,Platform,Genre,Publisher,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,7,New Super Mario Bros.,DS,Platform,Nintendo,9.23,6.50,2.90,30.01
1,373,Donkey Kong Country 3: Dixie Kong's Double Tro...,SNES,Platform,Nintendo,0.50,1.75,0.08,3.51
2,70,Gran Turismo 2,PS,Racing,Sony Computer Entertainment,3.42,1.69,0.50,9.49
3,1326,4 Nin uchi Mahjong,NES,Misc,Nintendo,0.00,1.45,0.00,1.45
4,1523,Romancing SaGa 3,SNES,Role-Playing,SquareSoft,0.00,1.29,0.01,1.30
...,...,...,...,...,...,...,...,...,...
538,9879,NBA 2K2,GC,Sports,Sega,0.02,0.00,0.00,0.12
539,881,Scooby Doo and the Cyber Chase,PS,Adventure,THQ,1.23,0.00,0.13,1.94
540,9897,Battles of Prince of Persia,DS,Strategy,Ubisoft,0.01,0.00,0.01,0.12
541,4571,NBA Street,GC,Sports,Electronic Arts,0.09,0.00,0.01,0.42


Используя OrdinalEncoder преобразуем данные.
Данное преобразование позволяет нам изменить данные в столбцах, которые ранее были типа object на численный. С помощью функции fit() получим данные из столбцов для дальнейшего изменения. Для столбца "Genre" используем тип кодирования "OneHotEncoder", так как при кодировании в нём появляется примерно равное количество единиц и нулей, а во всех других столбцах единица повторяется всего от 1 до 3 раз. Если так не сделать, будет ошибка с формированием древа, так как будут присутствовать нулевые элементы. 

In [49]:
OE = OrdinalEncoder()
df[['Name', 'Platform', 'Publisher']] = OE.fit_transform(df[['Name', 'Platform', 'Publisher']])
OHE = OneHotEncoder()
df['Genre'] = OHE.fit_transform(df[['Genre']]).toarray()
df

,Rank,Name,Platform,Genre,Publisher,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,7,325.0,3.0,0.0,95.0,9.23,6.50,2.90,30.01
1,373,117.0,18.0,0.0,95.0,0.50,1.75,0.08,3.51
2,70,182.0,11.0,0.0,115.0,3.42,1.69,0.50,9.49
3,1326,1.0,9.0,0.0,95.0,0.00,1.45,0.00,1.45
4,1523,382.0,18.0,0.0,120.0,0.00,1.29,0.01,1.30
...,...,...,...,...,...,...,...,...,...
538,9879,305.0,6.0,0.0,113.0,0.02,0.00,0.00,0.12
539,881,397.0,11.0,0.0,124.0,1.23,0.00,0.13,1.94
540,9897,35.0,3.0,0.0,132.0,0.01,0.00,0.01,0.12
541,4571,310.0,6.0,0.0,38.0,0.09,0.00,0.01,0.42


Мультиклассовая классификация.
Для примера мультиклассовой классификацией возьмем признак Genre.

In [50]:
Y = df["Genre"]
X = df.drop(columns = "Genre")

kf = KFold(n_splits=5, shuffle=False, random_state=None)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

Разделим строки на подстроки. 

In [51]:
kf.split(df)

<generator object _BaseKFold.split at 0x000002356B05B880>

Обучить 2-4 модель мультиклассовой классификацией (на выбор) на основе Kfold разбивки данных;
Выбрала способы - к-ближайших соседей и дерева решений.

In [52]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

Теперь разделим все по отдельности. Записываем данные в способы обучения. 

In [53]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()

knc.fit(X_train_kfold,Y_train_kfold)
predict_knc = knc.predict(X_test_kfold)

In [54]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

dtc = dtc.fit(X_train_kfold,Y_train_kfold)
predict_dtc = dtc.predict(X_test_kfold)

Вызываем данные. 

In [55]:
for train_index, test_index in kf.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_kfold = X.iloc[train_index]
    X_test_kfold = X.iloc[test_index]
    Y_train_kfold = Y.iloc[train_index]
    Y_test_kfold = Y.iloc[test_index]

    knc.fit(X_train_kfold,Y_train_kfold)
    predict_knc = knc.predict(X_test_kfold)

    dtc = dtc.fit(X_train_kfold,Y_train_kfold)
    predict_dtc = dtc.predict(X_test_kfold)


TRAIN: [109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306
 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342
 343 344 345 346 347 348 349 350 351 352 353

Теперь выведем результат моделей. 

In [57]:
    print("\n")
    print("Набор предсказаний дерева решений: ",predict_dtc)
    print("Набор предсказаний К-ближайших соседей: ",predict_knc)



Набор предсказаний дерева решений:  [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Набор предсказаний К-ближайших соседей:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Перейдем к числовой переменной и модели регрессии. 

Для примера регрессии возьмем признак Rank.

In [58]:
from sklearn.model_selection import KFold
Y = df["Rank"]
X = df.drop(columns = "Rank")

kf = KFold(n_splits=5, shuffle=False, random_state=None)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

Разделим строки на подстроки. 

In [59]:
kf.split(df)

<generator object _BaseKFold.split at 0x000002356C660CA0>

Обучить 2-4 модель регрессии (на выбор) на основе Kfold разбивки данных;
Выбрала способы - Обучение и предсказание Линейной регрессии, случайного леса и дерева решений.

In [60]:
from sklearn.linear_model import LinearRegression
lrc = LinearRegression()
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()

Теперь разделим все по отдельности. Записываем данные в способы обучения. 

In [61]:
from sklearn.linear_model import LinearRegression
lrc = LinearRegression()
lrc.fit(X_train_kfold,Y_train_kfold)
predict_lrc = lrc.predict(X_test_kfold)

In [62]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train_kfold,Y_train_kfold)
predict_dtr = dtr.predict(X_test_kfold)

In [63]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train_kfold,Y_train_kfold)
predict_rfr = rfr.predict(X_test_kfold)

Вызываем данные. 

In [64]:
for train_index, test_index in kf.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_kfold = X.iloc[train_index]
    X_test_kfold = X.iloc[test_index]
    Y_train_kfold = Y.iloc[train_index]
    Y_test_kfold = Y.iloc[test_index]

    lrc.fit(X_train_kfold,Y_train_kfold)
    predict_lrc = lrc.predict(X_test_kfold)

    dtr.fit(X_train_kfold,Y_train_kfold)
    predict_dtr = dtr.predict(X_test_kfold)

    rfr.fit(X_train_kfold,Y_train_kfold)
    predict_rfr = rfr.predict(X_test_kfold)

TRAIN: [109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306
 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342
 343 344 345 346 347 348 349 350 351 352 353

Теперь выведем результат моделей. 

In [67]:
    print("\n")
    print("Набор предсказаний регрессии дерева решений: ",predict_dtr)
    print("Набор предсказаний регрессии случайного леса: ",predict_rfr)
    print("Набор предсказаний линейной регрессии: ",predict_lrc)




Набор предсказаний регрессии дерева решений:  [ 6459.  4253. 12935. 10990. 10881. 13370. 12930. 10619. 12043. 11430.
 11598. 12154. 15447. 13997. 16586. 12211. 10249.  2921.  1539.  8614.
 13219. 12935.  9491. 13568. 10990. 12935. 13875. 15495.  3439.  6311.
 10039.  9613.  2236. 15242. 15188. 16170. 12804.  5611.  4889.   801.
  2074. 10427. 15495.  4889. 15939.  4849. 14639. 10794.  3546. 16103.
 15447.  2865. 13460.  8822.  9613.  4253. 15179. 12043.  1797. 10794.
  4388. 11108. 15005.  6931. 14815. 13997. 14796. 14241. 13742. 11526.
 13370.  6040.  6629.  5524. 15495. 14275. 10025.  6135.  4501.  7680.
  8064.  3439.  6311.  6442.  3439. 12935.  7672.  2074. 12305. 12154.
  8531.  3375. 11108. 10471.  9029.  7451.  1797.  9382. 12211.  4437.
 12706.  5906.  8504.  9844.   721.  9967.  4537.  7277.]
Набор предсказаний регрессии случайного леса:  [ 6503.64  4353.79 13137.05 11067.75 10954.17 13249.25 13202.61 10544.45
 11927.72 11495.9  11563.21 12000.56 15543.82 13793.71 16316.44 

In [68]:
print("Набор предсказаний линейной регрессии: ",predict_lrc)

Набор предсказаний линейной регрессии:  [ 7488.16114218  7893.0081176   9033.93909537  7889.89967184
  8940.43786091  9206.8897576   8120.87103636 10394.03108622
  8943.37097279  8388.48231258  9938.9311741   9207.94485873
  9263.95210144  8906.80183473  9715.3225449   8411.18275589
  7633.93845009  9124.58767148  5803.2846942   8715.75307245
  9805.60348137 10246.83179984  9856.79522349  9186.77359279
  9313.24313283  9067.062452    9977.85073125  9461.57053698
  8652.55984501  7980.4029029   8016.72909196  8596.38612174
  6994.12372937 10488.46975065  7960.2414422  10214.18867193
  8565.79252506 10049.73567454  8303.35756293  4187.3303015
  6574.83630848  9980.71774586  7875.41381213  7518.50426357
  8935.1514383   8830.67919555  8441.14513996  7706.73247377
  8178.96327336  9400.31231091  9395.4932057   6804.15823482
  9040.56671839  9585.84828681  9565.49656084  8687.31921645
  8437.98554848  8913.49695957  7724.8148005   7887.63316746
  7032.07720271  9937.09668394  9833.32853116 

Вывод: В ходе данной практической работы, был изучен материал на тему "Классификация и регрессия", а также было обучено 2-4 модели мультиклассовой классификацией и регрессии.